## Databases 

In [1]:
import os
import pandas as pd
import time

In [2]:
hostname = "127.0.0.1"

In [23]:
hospital = pd.read_json("../data/testData/hospital-sort.json").to_dict("records") 
hospitalAllPaths = pd.read_json("../data/testData/hospitall-allPaths.json").to_dict("list")

In [8]:
college = pd.read_json("../data/full/college-tab.json").to_dict("records")

### Mongo 

In [ ]:
from pymongo import MongoClient

In [344]:
client = MongoClient(host=hostname,port=27017)

In [345]:
client

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True)

In [ ]:
[[e.update(id=idx+1) for idx, e in enumerate(data)] for data in [localHospital,bigHospital,college]]

In [ ]:
hospitalMongo = client["hospital"]["hospital"]
collegeMongo = client["college"]["college"]

In [ ]:
hospitalMongo.insert_many(localHospital)
hospitalMongo.insert_many(bigHospital)
collegeMongo.insert_many(college)

### Neo4j 

In [347]:
#from neo4j import GraphDatabase

In [8]:
from py2neo import Graph, Node, Relationship

In [34]:
txpool = []
for e in hospital:
    start = Node("Hospital",name=e["start_name"],lat=e["start_lat"],lon=e["start_lon"])
    end = Node("Hospital",name=e["end_name"],lat=e["end_lat"],lon=e["end_lon"])
    
    r = Relationship(start,"NEAREST_PATH",end)
    txpool.append(r)
    
    for i in hospitalAllPaths.get(e["start_name"]): 
        #
        destination = Node("Hospital", name=i["end_name"], lat=e["end_lat"], lon=e["end_lon"]) 
        is_connected = Relationship(start, "IS_CONNECTED", destination, distance=i["distance"])
        txpool.append(is_connected)
    else: 
        continue 

In [13]:
unique = set([e["Name"] for e in college])

for idx, collegeName in enumerate(unique):
    # Filter 
    pred = list(filter(lambda e: e["Name"] == collegeName, college)) 
    getProgramName = list(map(lambda e: Node("ProgramName",name=e["ProgramName"]), pred))
                              
    university = Node("University", name=pred[0]["Name"], web=pred[0]["Web"], city=pred[0]["City"])    
    
    [txpool.append(Relationship(e,"HAS_CAPACITY",university)) for e in getProgramName]

0 - University of Pittsburgh - Bradford - 2 - 2
1 - Spencerian College - Louisville - 5 - 5
2 - Moraine Valley Community College - 4 - 4
3 - Bucks County Community College - 5 - 5
4 - Orlando VA Medical Center - 1 - 1
5 - East Central College - 6 - 6
6 - Belmont Abbey College - 1 - 1
7 - Madonna University - 7 - 7
8 - Appalachian State University - 11 - 11
9 - Texas A&M University - Commerce - 3 - 3
10 - West Kentucky Community and Technical College - 4 - 4
11 - Logan University - 1 - 1
12 - Longy School of Music - 1 - 1
13 - VA Southern Oregon Rehabilitation Center - 3 - 3
14 - Florida Agricultural and Mechanical University - 10 - 10
15 - Fashion Institute of Technology - 1 - 1
16 - Jung Tao School of Classical Chinese Medicine - 1 - 1
17 - VA Greater Los Angeles - Sepulveda Ambulatory Care Center - 3 - 3
18 - Chattahoochee Technical College - 3 - 3
19 - University of Detroit Mercy - 13 - 13
20 - Edgecombe Community College - 3 - 3
21 - Anthem College - Bryman School - 6 - 6
22 - Sout

252 - University of West Georgia - 7 - 7
253 - Arizona School of Acupuncture and Oriental Medicine - 2 - 2
254 - Sanford-Brown College at Jacksonville - 10 - 10
255 - Minneapolis Community and Technical College - 3 - 3
256 - Brown University - 5 - 5
257 - Niagara University - 3 - 3
258 - Erwin Technical College - 4 - 4
259 - Dartmouth College - 5 - 5
260 - University of Tennessee Medical Center at Knoxville - 2 - 2
261 - Mercy College - 13 - 13
262 - Northampton Community College - 5 - 5
263 - The Diller-Quaile School of Music - 1 - 1
264 - Pickens Technical College - 1 - 1
265 - Mid-Del Technology Center - 1 - 1
266 - North Dakota State College of Science - 4 - 4
267 - Francis Tuttle Technology Center - Rockwell - 1 - 1
268 - Five Branches University:  Graduate School of Traditional Chinese Medicine - Santa Cruz, CA - 2 - 2
269 - Evergreen Valley College - 1 - 1
270 - Reid Hospital and Health Care Services - 1 - 1
271 - National Park College - 2 - 2
272 - Kennett Career and Technology

513 - Coppin State University - 6 - 6
514 - Bloomsburg University of Pennsylvania - 10 - 10
515 - CVPH Medical Center School of Radiologic Technology - 1 - 1
516 - Saint Augustine's University - 1 - 1
517 - Paradise Valley Unified School District - 1 - 1
518 - Coney Island Hospital - 1 - 1
519 - Henderson State University - 5 - 5
520 - Delaware State University - 7 - 7
521 - Northwest Mississippi Community College - 2 - 2
522 - Los Angeles Harbor College - 1 - 1
523 - City College - Altamonte Springs - 1 - 1
524 - University of Minnesota - Duluth - 4 - 4
525 - University of Minnesota - Crookston - 1 - 1
526 - South University - 27 - 27
527 - Colorado State University - 14 - 14
528 - Western New England University - 3 - 3
529 - University of Tulsa - 8 - 8
530 - Genesee Valley BOCES Batavia Campus - 1 - 1
531 - University of North Carolina at Wilmington - 7 - 7
532 - Notre Dame College - 2 - 2
533 - Loyola University of Chicago - 16 - 16
534 - Waukesha County Technical College - 3 - 3
53

767 - International Business College - Indianapolis - 1 - 1
768 - Salt Lake Community College - 7 - 7
769 - Franciscan Missionaries of Our Lady University - 7 - 7
770 - Southwest College of Naturopathic Medicine & Health Sciences - 1 - 1
771 - Clovis Community College - 3 - 3
772 - Eastern Connecticut State University - 1 - 1
773 - Franklin University - 1 - 1
774 - Holy Family University - 4 - 4
775 - Saint Peter's University - 8 - 8
776 - Sacred Heart University - 9 - 9
777 - North Idaho College - 3 - 3
778 - Westmoreland County Community College - 4 - 4
779 - Allen College - 7 - 7
780 - The University of Texas Rio Grande Valley - 8 - 8
781 - College of DuPage - 8 - 8
782 - VA Maryland Health Care System (Baltimore/Perry Point) - 5 - 5
783 - Reading Area Community College - 2 - 2
784 - University of Saint Francis - Fort Wayne - 9 - 9
785 - Kutztown University of Pennsylvania - 4 - 4
786 - Solex College - 2 - 2
787 - Grand Canyon University - 3 - 3
788 - Mount Saint Mary's University -

1013 - Marygrove College - 3 - 3
1014 - New Mexico Highlands University - 2 - 2
1015 - University of Virginia - 20 - 20
1016 - Pima Community College - 9 - 9
1017 - New York Academy of Art - 1 - 1
1018 - Virginia Tech University Falls Church - 1 - 1
1019 - University of Iowa - 39 - 39
1020 - Hawaii College of Oriental Medicine - 2 - 2
1021 - Worcester State University - 9 - 9
1022 - Syracuse University - 15 - 15
1023 - Center for Instruction, Technology, and Innovation (CiTi) - 1 - 1
1024 - Finlandia University - 2 - 2
1025 - Carondelet Health - 2 - 2
1026 - Eastern Michigan University - 10 - 10
1027 - Western Technical College - 8 - 8
1028 - Jefferson-Lewis-Hamilton-Herkimer-Oneida BOCES - 2 - 2
1029 - Florida State College at Jacksonville - 9 - 9
1030 - Onondaga Community College - 4 - 4
1031 - Louisiana State University at Shreveport - 1 - 1
1032 - Vanderbilt University - 21 - 21
1033 - Lakeshore Technical College - 4 - 4
1034 - Five Branches University:  Graduate School of Traditio

...

1765 - Tennessee Technological University - 5 - 5
1766 - Jefferson County Dubois Area Vocational Technical Practical Nursing Program - 1 - 1
1767 - Langston University - 5 - 5
1768 - Huntingdon County Career and Technology Center - 1 - 1
1769 - Ohio Northern University - 5 - 5
1770 - Akron School of Practical Nursing - 1 - 1
1771 - Murray State University - 11 - 11
1772 - Gulf Coast Veterans Health Care System - 1 - 1
1773 - Adrian College - 1 - 1
1774 - American Sentinel University - 3 - 3
1775 - Salem VA Medical Center - 3 - 3
1776 - New Britain School of Nurse Anesthesia - 1 - 1
1777 - San Juan College - 4 - 4
1778 - Gaston College - 3 - 3
1779 - Ashland University - 11 - 11
1780 - U. S. Army Medical Department Center and School - 7 - 7
1781 - Grambling State University - 5 - 5
1782 - Mississippi University for Women - 10 - 10
1783 - SUNY Polytechnic Institute - 3 - 3
1784 - California State University - Los Angeles - 11 - 11
1785 - Aiken Technical College - 4 - 4
1786 - Florida Col

2006 - Cochise College - 1 - 1
2007 - American Harbor College - 1 - 1
2008 - Southern Union State Community College - 2 - 2
2009 - Southwestern Oklahoma State University - 7 - 7
2010 - Heart of America Psychology Training Consortium - 1 - 1
2011 - Beth Israel Deaconess Medical Center - 3 - 3
2012 - Western Nevada College - 1 - 1
2013 - University of California, Irvine - 8 - 8
2014 - Vermont Technical College - 2 - 2
2015 - SUNY Downstate Medical Center - 12 - 12
2016 - Sonoma State University - 5 - 5
2017 - Hope College - 6 - 6
2018 - University of Colorado Denver - 26 - 26
2019 - Mid Michigan Community College - 3 - 3
2020 - SUNY College at New Paltz - 8 - 8
2021 - Teachers College of Columbia University - 6 - 6
2022 - American Public University System - 1 - 1
2023 - Pueblo Community College - 6 - 6
2024 - Triton College - 3 - 3
2025 - Cambridge College - 1 - 1
2026 - School of Visual Arts - 2 - 2
2027 - Astor Services for Children and Families - 1 - 1
2028 - VA NY Harbor Healthcare S

2245 - Northwestern University - 16 - 16
2246 - Mary Baldwin College - 4 - 4
2247 - Dodge City Community College - 2 - 2
2248 - JRMC School of Nursing - 2 - 2
2249 - Panola College - 2 - 2
2250 - Darton State College - 6 - 6
2251 - St Joseph's Hospital - 1 - 1
2252 - Oberlin College - 1 - 1
2253 - University of Texas Southwestern Medical Center - 10 - 10
2254 - Messiah College - 8 - 8
2255 - University of Maine at Fort Kent - 1 - 1
2256 - Milwaukee School of Engineering - 2 - 2
2257 - Fitchburg State University - 3 - 3
2258 - Oklahoma Baptist University - 5 - 5
2259 - Quincy University - 1 - 1
2260 - Bellin College - 3 - 3
2261 - Mattia College - 1 - 1
2262 - Southern State Community College - 1 - 1
2263 - Everest College - Colorado Springs - 2 - 2
2264 - Redlands Community College - 1 - 1
2265 - Cumberland University - 3 - 3
2266 - Atlantic Union College - 3 - 3
2267 - POLY Languages Institute - 3 - 3
2268 - Linn-Benton Community College - 2 - 2
2269 - Elmhurst College - 4 - 4
2270 - 

2497 - Windham Community Memorial Hospital - 1 - 1
2498 - Antelope Valley College - 1 - 1
2499 - The University of Tennessee - Martin - 5 - 5
2500 - York Technical College - 5 - 5
2501 - Saint Michaels College - 1 - 1
2502 - Winona State University - 6 - 6
2503 - William Penn University - 1 - 1
2504 - Dongguk University - Los Angeles - 3 - 3
2505 - Meharry Medical College - 4 - 4
2506 - South Carolina State University - 6 - 6
2507 - University of Arizona - 21 - 21
2508 - El Paso Community College - 7 - 7
2509 - Albany Technical College - 2 - 2
2510 - Fairfield University - 4 - 4
2511 - Brookdale Community College - 2 - 2
2512 - Los Angeles Pierce College - 1 - 1
2513 - McCann School of Business and Technology - 3 - 3
2514 - VA Eastern Kansas Health Care System - 1 - 1
2515 - Pima Medical Institute - 31 - 31
2516 - Washington Bible College and Capital Bible Seminary - 2 - 2
2517 - University of Toledo - 22 - 22
2518 - Texas Christian University - 9 - 9
2519 - Presbyterian Hospital - 1 -

2735 - Angelina College - 1 - 1
2736 - Southeast Louisiana Veterans Health Care System - 2 - 2
2737 - Concordia University, St. Paul - 3 - 3
2738 - Clearfield Hospital - 1 - 1
2739 - Texas Tech University Health Sciences Center - 20 - 20
2740 - Coralville VA Outpatient Clinic - 1 - 1
2741 - Terra State Community College - 2 - 2
2742 - Blackstone Valley Regional Vocational Technical High School - 1 - 1
2743 - Shelton State Community College - 2 - 2
2744 - Jefferson Community College - 2 - 2
2745 - Hannibal-LaGrange University - 4 - 4
2746 - Christopher Newport University - 2 - 2
2747 - Fuller Theological Seminary - 2 - 2
2748 - Santa Ana College - 2 - 2
2749 - Charleston Area Medical Center- School of Nursing Anesthesia - 1 - 1
2750 - University of Massachusetts - Lowell - 8 - 8
2751 - North Seattle Community College - 1 - 1
2752 - South Florida State College - 4 - 4
2753 - Fort Scott Community College - 1 - 1
2754 - Wayne State University - 24 - 24
2755 - Northwest Florida State Colleg

In [35]:
import time
graph = Graph(f"bolt://{hostname}:7687", auth=("neo4j", "12345678"))
tx = graph.begin()
start_time = time.time() 
[print(f"Processing {idx} out of {len(txpool)} - {tx.create(e)}") for idx, e in enumerate(txpool)]
print(f"Before commit: {time.time() - start_time}")
graph.commit(tx)
print(f"After commit: {time.time() - start_time}")

Processing 0 out of 100 - None
Processing 1 out of 100 - None
Processing 2 out of 100 - None
Processing 3 out of 100 - None
Processing 4 out of 100 - None
Processing 5 out of 100 - None
Processing 6 out of 100 - None
Processing 7 out of 100 - None
Processing 8 out of 100 - None
Processing 9 out of 100 - None
Processing 10 out of 100 - None
Processing 11 out of 100 - None
Processing 12 out of 100 - None
Processing 13 out of 100 - None
Processing 14 out of 100 - None
Processing 15 out of 100 - None
Processing 16 out of 100 - None
Processing 17 out of 100 - None
Processing 18 out of 100 - None
Processing 19 out of 100 - None
Processing 20 out of 100 - None
Processing 21 out of 100 - None
Processing 22 out of 100 - None
Processing 23 out of 100 - None
Processing 24 out of 100 - None
Processing 25 out of 100 - None
Processing 26 out of 100 - None
Processing 27 out of 100 - None
Processing 28 out of 100 - None
Processing 29 out of 100 - None
Processing 30 out of 100 - None
Processing 31 out 

In [24]:
graph.run("MATCH (n) RETURN COUNT(n)")

COUNT(n)
158


In [12]:
#graph.run("MATCH (n) DETACH DELETE(n)")

(No data)